# Imports

In [1021]:
import datetime
import math
import numpy as np
import pandas as pd
import random
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
from re import match

### Remove Repeat Values

In [1022]:
g = pd.read_csv('fixed_colleges.csv',names=['Name','College_y']).reset_index().drop('index',axis=1).iloc[1:]
g.head()

,Name,College_y
1,"Abdul-Wahad,Tariq",Michigan (1993–1994)San Jose State (1995–1997)
2,"Abdur-Rahim,Shareef",California (1995–1996)
3,"Alexander,Cory",Virginia (1991–1995)
4,"Allen,Ray",UConn (1993–1996)
5,"Aluma,Peter",Liberty (1993–1997)


In [1023]:
nba_stats = pd.read_csv('nba_player_bio_raw.csv')
nba_stats = pd.merge(nba_stats,g,on='Name')
nba_stats.head()

,Unnamed: 0,Year,Name,Born,Nationality,Height,Weight,Highschool,College,Draft,...,FirstName,LastName,Birthday,Birthplace,American,CareerStart,CareerEnd,Height (in),Weight (lbs),College_y
0,0,1999.0,"Abdul-Wahad,Tariq","['(1974-11-03) November 3, 1974 ', 'Maisons-Al...",NaN,6 ft 6 in (1.98 m),223 lb (101 kg),"Aristide Briand (Évreux, France)",Michigan (1993–1994)San Jose State (1995–1997),1997.0,...,Tariq,Abdul-Wahad,1974-11-03,"Maisons-Alfort, Val-de-Marne, France",0.0,1997.0,2003.0,78.0,223.0,Michigan (1993–1994)San Jose State (1995–1997)
1,440,2000.0,"Abdul-Wahad,Tariq","['(1974-11-03) November 3, 1974 ', 'Maisons-Al...",NaN,6 ft 6 in (1.98 m),223 lb (101 kg),"Aristide Briand (Évreux, France)",Michigan (1993–1994)San Jose State (1995–1997),1997.0,...,Tariq,Abdul-Wahad,1974-11-03,"Maisons-Alfort, Val-de-Marne, France",0.0,1997.0,2003.0,78.0,223.0,Michigan (1993–1994)San Jose State (1995–1997)
2,880,2001.0,"Abdul-Wahad,Tariq","['(1974-11-03) November 3, 1974 ', 'Maisons-Al...",NaN,6 ft 6 in (1.98 m),223 lb (101 kg),"Aristide Briand (Évreux, France)",Michigan (1993–1994)San Jose State (1995–1997),1997.0,...,Tariq,Abdul-Wahad,1974-11-03,"Maisons-Alfort, Val-de-Marne, France",0.0,1997.0,2003.0,78.0,223.0,Michigan (1993–1994)San Jose State (1995–1997)
3,1320,2002.0,"Abdul-Wahad,Tariq","['(1974-11-03) November 3, 1974 ', 'Maisons-Al...",NaN,6 ft 6 in (1.98 m),223 lb (101 kg),"Aristide Briand (Évreux, France)",Michigan (1993–1994)San Jose State (1995–1997),1997.0,...,Tariq,Abdul-Wahad,1974-11-03,"Maisons-Alfort, Val-de-Marne, France",0.0,1997.0,2003.0,78.0,223.0,Michigan (1993–1994)San Jose State (1995–1997)
4,1760,2003.0,"Abdul-Wahad,Tariq","['(1974-11-03) November 3, 1974 ', 'Maisons-Al...",NaN,6 ft 6 in (1.98 m),223 lb (101 kg),"Aristide Briand (Évreux, France)",Michigan (1993–1994)San Jose State (1995–1997),1997.0,...,Tariq,Abdul-Wahad,1974-11-03,"Maisons-Alfort, Val-de-Marne, France",0.0,1997.0,2003.0,78.0,223.0,Michigan (1993–1994)San Jose State (1995–1997)


In [1024]:
#Scraper Messed Up Players, so Scraped and Deleted Birthdates, so Rescraped Every Row with Birth Dates
nba_stats = nba_stats.drop('Unnamed: 0',axis=1)

In [1025]:
nba_stats_copy = nba_stats

### Adjusting Column DataTypes 

In [1026]:
nba_stats_copy.dtypes

Year            float64
Name             object
Born             object
Nationality      object
Height           object
Weight           object
Highschool       object
College          object
Draft           float64
CareerYears      object
History          object
Awards           object
FirstName        object
LastName         object
Birthday         object
Birthplace       object
American        float64
CareerStart     float64
CareerEnd       float64
Height (in)     float64
Weight (lbs)    float64
College_y        object
dtype: object

In [1027]:
nba_stats_copy["Nationality"] = nba_stats_copy["Nationality"].apply(lambda x: None if isinstance(x,float) else x)
nba_stats_copy["Weight"] = nba_stats_copy["Weight"].apply(lambda x: None if isinstance(x,float) else x)
nba_stats_copy["Height"] = nba_stats_copy["Height"].apply(lambda x: None if isinstance(x,float) else x)
nba_stats_copy["Highschool"] = nba_stats_copy["Highschool"].apply(lambda x: None if isinstance(x,float) else x)
nba_stats_copy["College"] = nba_stats_copy["College"].apply(lambda x: None if isinstance(x,float) else x)

# Cleaned Values for Table:
### - Columns (Which is Manipulated)
    - First Name (Name)
    - Last Name (Name)
    - Year
    - Nationality
    - Birthday (Born)
    - Birthplace (Born)
    - Born in America? (Born)
    - Career Start (CareerYrs)
    - Career End (CareerYrs)
    - Height ins (Height)
    - Weight lbs (Weight)
    - Highschools (Highschool)
    - Highschool Cities and State (Highschool)
    - Highschool in the United States? (Highschool)
    - Colleges (College)
    - College Years (College)
    - Which Teams and What Years They Played For (History)

### First Name, Last Name

In [1028]:
#Separate Name into First and Last Components
nba_stats_copy['FirstName'] = nba_stats_copy["Name"].apply(lambda x: x.split(',')[1])
nba_stats_copy['LastName'] = nba_stats_copy["Name"].apply(lambda x: x.split(',')[0])

### Birthday, Birthplace

In [1029]:
def getBday(x):
    
    pattern_bday = re.compile(r"\d\d\d\d-\d\d-\d\d", re.IGNORECASE)
    
    if pattern_bday.search(str(x)) != None:
        return pattern_bday.search(str(x)).group(0)
    else:
        return None

In [1030]:
#Convert Birthdays to Datetime and Create Separate Column
nba_stats_copy['Birthday'] = nba_stats_copy['Born'].apply(lambda x: None if x == None else getBday(x))
nba_stats_copy['Birthday'] = pd.to_datetime(nba_stats_copy['Birthday'])

In [1031]:
def getBirthPlace(x):
    
    if isinstance(x,float):
        return None
    
    pattern = re.compile(r"'(.*?)'",re.IGNORECASE)
    matches = pattern.findall(x)
    
    if len(matches) == 2:
        return matches[1]
    else:
        matches = re.split('\d\d\d\d',x.replace("']",""))
        return matches[-1]

In [1032]:
nba_stats_copy["Birthplace"] = nba_stats_copy['Born'].apply(lambda x: None if x == None else getBirthPlace(x))

### American?

In [1033]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California",
               "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida",
               "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas",
               "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota",
               "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire",
               "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
               "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah",
               "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [1034]:
def bornInUS(x):
    if 'U.S.' in x:
        return 1
    else:
        for item in x.split(', '):
            if item in state_names:
                return 1
        return 0

In [1035]:
nba_stats_copy['American'] = nba_stats_copy['Birthplace'].apply(lambda x: None if x == None else bornInUS(x))

In [1036]:
#Fill in Nationality as American if We Know They Were Born in America
all_american = nba_stats_copy[nba_stats_copy["American"] == 1].index

In [1037]:
nba_stats_copy.loc[all_american,'Nationality'] = 'American'

### Career Start, Career End

In [1038]:
def StartPlaying(x):
    if isinstance(x,float):
        return None
    else:
        return x.split('–')[0][-4:]

In [1039]:
def EndPlaying(x):
    if isinstance(x,float):
        return None
    else:
        if "present" in x:
            return datetime.date.today().year
        else:
            return x.split('–')[-1][-4:]

In [1040]:
#Separate CareerYears into Start and End Years for Career
nba_stats_copy["CareerStart"] = nba_stats_copy['CareerYears'].apply(lambda x: None if x == None else StartPlaying(x))
nba_stats_copy["CareerEnd"] = nba_stats_copy['CareerYears'].apply(lambda x: None if x == None else EndPlaying(x))

In [1041]:
nba_stats_copy[['Year','Draft','CareerEnd','CareerStart']] = nba_stats_copy[['Year','Draft','CareerEnd','CareerStart']].astype('float')

### Height, Weight

In [1042]:
def getHeight(x):
    
    if x is None:
        return None
    
    pattern_ht = re.compile(r"\d{1,2} ft \d{1,2} in", re.IGNORECASE)
    
    if pattern_ht.match(str(x)) != None:
        
        spl_lst = pattern_ht.match(str(x)).group(0).split('ft')
        feet_to_inch = int(spl_lst[0].strip()) * 12
        inches = int(spl_lst[1].replace('in','').strip())
        
        return feet_to_inch + inches
    else:
        return None

In [1043]:
def getWeight(x):
    
    if x is None:
        return None
    
    pattern_ht = re.compile(r"\d{1,3} lb", re.IGNORECASE)
    
    if pattern_ht.match(str(x)) != None:
        
        lbs = pattern_ht.match(str(x)).group(0).replace('lb','')
        
        return int(lbs.strip())
    else:
        return None

In [1044]:
nba_stats_copy['Height (in)'] = nba_stats_copy['Height'].apply(lambda x: None if x is None else getHeight(x))
nba_stats_copy['Weight (lbs)'] = nba_stats_copy['Weight'].apply(lambda x: None if x is None else getWeight(x))

### High Schools, High School Cities and States 

In [1045]:
def getHighSchool(x):
    
    if x is None:
        return None
    
    else:
        x = re.sub('\d{1-3}', '', x)
        x = x.strip('[]').split("', ")
        x = [re.sub(r'\[\d{1,3}\]','',i).strip("''") for i in x]
        x = [re.sub(r'\([^)]*, [^)]*\)','',i) for i in x]
            
        
        if len(x) == 1:
            return x[0], None, None, None
        if len(x) == 2:
            return x[0], x[1], None, None
        if len(x) == 3:
            return x[0], x[1], x[2], None
        if len(x) == 4:
            return x[0], x[1], x[2], x[3]
        else:
            return None, None, None, None

In [1046]:
a = nba_stats_copy['Highschool'].apply(lambda x: None if x is None else getHighSchool(x)[0])
b = nba_stats_copy['Highschool'].apply(lambda x: None if x is None else getHighSchool(x)[1])
c = nba_stats_copy['Highschool'].apply(lambda x: None if x is None else getHighSchool(x)[2])
d = nba_stats_copy['Highschool'].apply(lambda x: None if x is None else getHighSchool(x)[3])

In [1047]:
nba_stats_copy = pd.concat([nba_stats_copy,a,b,c,d],axis=1)
nba_stats_copy.columns.values[-4:] = ['HS1', 'HS2', 'HS3', 'HS4']

In [1048]:
def getHSCity(x):
    
    if x is None:
        return None, None, None, None
    
    pattern = re.compile(r'\([^)]*, [^)]*\)',re.IGNORECASE)
    
    if pattern.findall(str(x)) != None:
        
        match = pattern.findall(str(x))
        
        if len(match) == 1:
            return match[0], None, None, None
        if len(match) == 2:
            return match[0], match[1], None, None
        if len(match) == 3:
            return match[0], match[1], match[2], None
        if len(match) == 4:
            return match[0], match[1], match[2], match[3]
        else:
            return None, None, None, None

In [1049]:
a = nba_stats_copy['Highschool'].apply(lambda x: None if isinstance(x,str)==False else getHSCity(x)[0])
b = nba_stats_copy['Highschool'].apply(lambda x: None if isinstance(x,str)==False else getHSCity(x)[1])
c = nba_stats_copy['Highschool'].apply(lambda x: None if isinstance(x,str)==False else getHSCity(x)[2])
d = nba_stats_copy['Highschool'].apply(lambda x: None if isinstance(x,str)==False else getHSCity(x)[3])

In [1050]:
nba_stats_copy = pd.concat([nba_stats_copy,a,b,c,d],axis=1)
nba_stats_copy.columns.values[-4:] = ['HSC1', 'HSC2', 'HSC3', 'HSC4']

In [1051]:
nba_stats_copy['HSC1'] = nba_stats_copy['HSC1'].apply(lambda x: None if x is None else x.strip('()'))
nba_stats_copy['HSC2'] = nba_stats_copy['HSC2'].apply(lambda x: None if x is None else x.strip('()'))
nba_stats_copy['HSC3'] = nba_stats_copy['HSC3'].apply(lambda x: None if x is None else x.strip('()'))
nba_stats_copy['HSC4'] = nba_stats_copy['HSC4'].apply(lambda x: None if x is None else x.strip('()'))

### Highschool in US 

In [1052]:
def inUS(x):
    place = x.split(', ')[-1]
    if place not in state_names:
        return False
    else:
        return True

In [1053]:
a = nba_stats_copy['HSC1'].apply(lambda x: None if x is None else inUS(x))
b = nba_stats_copy['HSC2'].apply(lambda x: None if x is None else inUS(x))
c = nba_stats_copy['HSC3'].apply(lambda x: None if x is None else inUS(x))
d = nba_stats_copy['HSC4'].apply(lambda x: None if x is None else inUS(x))

In [1054]:
instates = []
for n in range(len(nba_stats_copy)):
    check = [a[n],b[n],c[n],d[n]]
    
    #If we find one place outside the US then automatically False
    if False in check:
        instates.append(0)
    else:
        #If we find no True then it is None since we eliminated all Falses already
        if True not in check:
            instates.append(None)
        #Since there are none that have Falses and we got rid of ones where they had no schools listed, the rest are true
        else:
            instates.append(1)

In [1055]:
nba_stats_copy['USHS'] = instates

# Colleges

In [1056]:
def getCleanCol(x):
    
    #Grab Whitespaces and Alphabetic Items
    pattern = re.compile(r"[^]^[^'^(^)]",re.IGNORECASE)
    s=''
    #Combine Those Items from above into string s
    if pattern.findall(str(x)) != None:
        for item in pattern.findall(str(x)):
            s+=item    
    
    #Split string into list at the years and remove years
    match = re.split('\d\d\d\d.\d\d\d\d',s)
    
    #Remove values from list which == ''
    match = list(filter(None,match))
    
    match = [i.strip() for i in match]
    match = [i.replace(', ','') for i in match]
            
    return match

In [1057]:
def getCollege(x):
    
    #Check null
    if isinstance(x,float):
        return None, None, None, None
    
    else:
        x = getCleanCol(x)
        
        if len(x) == 1:
            return x[0], None, None, None
        if len(x) == 2:
            return x[0], x[1], None, None
        if len(x) == 3:
            return x[0], x[1], x[2], None
        if len(x) == 4:
            return x[0], x[1], x[2], x[3]
        else:
            return None, None, None, None


In [1058]:
nba_stats_copy['College1'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollege(x)[0])
nba_stats_copy['College2'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollege(x)[1])
nba_stats_copy['College3'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollege(x)[2])
nba_stats_copy['College4'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollege(x)[3])

In [1059]:
def getCollegeYrs(x):
    
    #Check null
    if isinstance(x,float):
        return None, None, None, None
    
    #Pattern
    pattern = re.compile(r'\d\d\d\d–\d\d\d\d',re.IGNORECASE)
    
    #If found something in x with pattern
    if pattern.findall(str(x)) != None:
        
        #Get list of matches
        match = pattern.findall(str(x))
        match = [i.replace('–','-') for i in match]
        
        if len(match) == 1:
            return match[0], None, None, None
        if len(match) == 2:
            return match[0], match[1], None, None
        if len(match) == 3:
            return match[0], match[1], match[2], None
        if len(match) == 4:
            return match[0], match[1], match[2], match[3]
        else:
            return None, None, None, None

In [1060]:
nba_stats_copy['CollegeY1'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollegeYrs(x)[0])
nba_stats_copy['CollegeY2'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollegeYrs(x)[1])
nba_stats_copy['CollegeY3'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollegeYrs(x)[2])
nba_stats_copy['CollegeY4'] = nba_stats_copy['College_y'].apply(lambda x: None if isinstance(x,str)==False else getCollegeYrs(x)[3])

### Fixing History Section 

In [1061]:
def getHist(x):
    
    if isinstance(x,float):
        return None
    x_list = x.replace('–','-').strip('][').split("\', ")
    x_list = [i.strip("'':").strip('""') for i in x_list]
    
    if len(x_list) == 1:
        return x_list[0]
    
    global a,b
    a,b = 0, len(x_list)
    
    for item in x_list:
        if "as player" in item.lower():
            a = x_list.index(item) + 1
        if "As coach" in item:
            b = x_list.index(item)
            break
        elif "career highlights" in item.lower():
            b = x_list.index(item)
            break
        else:
            b = len(x_list)

    return x_list[a:b]

In [1062]:
nba_stats_copy['Teams'] = nba_stats_copy['History'].apply(lambda x: None if x is None else getHist(x))

# Final Cleaned DataFrame

In [1063]:
final_cleaned = nba_stats_copy[['Year','FirstName','LastName','Name','Nationality','American','Draft','Birthday',
                'Birthplace', 'CareerStart', 'CareerEnd','Height (in)', 'Weight (lbs)',
                'HS1', 'HS2', 'HS3', 'HS4', 'HSC1','HSC2', 'HSC3', 'HSC4', 'USHS',
                'CollegeY1', 'CollegeY2', 'CollegeY3','CollegeY4', 'College1', 'College2',
                'College3', 'College4',"Teams"]].sort_values('Year')
final_cleaned.head()

,Year,FirstName,LastName,Name,Nationality,American,Draft,Birthday,Birthplace,CareerStart,...,USHS,CollegeY1,CollegeY2,CollegeY3,CollegeY4,College1,College2,College3,College4,Teams
0,1999.0,Tariq,Abdul-Wahad,"Abdul-Wahad,Tariq",None,0.0,1997.0,1974-11-03,"Maisons-Alfort, Val-de-Marne, France",1997.0,...,0.0,1993-1994,1995-1997,None,None,Michigan,San Jose State,None,None,"[1997-1999Sacramento Kings, 1999-2000Orlando M..."
1651,1999.0,Tyrone,Nesby,"Nesby,Tyrone",American,1.0,1998.0,1976-01-31,"Cairo, Illinois",1998.0,...,1.0,1994-1996,1996-1998,None,None,Vincennes,UNLV,None,None,"[1998-1999Sioux Falls Skyforce, 1999-2000Los A..."
1635,1999.0,Steve,Nash,"Nash,Steve",Canadian,0.0,1996.0,1974-02-07,"Johannesburg, South Africa",1996.0,...,0.0,1992-1996,None,None,None,Santa Clara,None,None,None,"[1996-1998Phoenix Suns, 1998-2004Dallas Maveri..."
450,1999.0,Marty,Conlon,"Conlon,Marty",American,1.0,1990.0,1968-01-19,"The Bronx, New York, U.S.",1990.0,...,1.0,1986-1990,None,None,None,Providence,None,None,None,"[1990Le Mans, 1990-1991Rockford Lightning, 199..."
1634,1999.0,Makhtar,N'Diaye,"N'Diaye,Makhtar",Senegalese,0.0,1998.0,1973-12-12,"Diourbel, Senegal",1998.0,...,1.0,1993-1995,1996-1998,None,None,Michigan,North Carolina,None,None,None


In [1064]:
final_cleaned.to_csv('C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\nba_personal_info_cleaned.csv')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\nba_personal_info_cleaned.csv'

In [ ]:
cleaned_info = pd.DataFrame(np.zeros((len(final_cleaned.T),2)),
                            final_cleaned.columns,
                            columns=['DataType','Description'])

## Descriptions for Cleaned Frame Columns

In [ ]:
descriptions = {'Year':'Basketball Season for the Player (eg. 2020 = 2019-2020 NBA Season)',
 'FirstName':"NBA Player's First Name",
 'LastName':"NBA Player's Last Name",
 'Name': "NBA Player's Full Name",
 'Nationality': "NBA Player's Nationality (if None, Not Found on Wikipedia)",
 'American': '1 if Player was Born in the United States',
 'Draft': 'Year Player was Drafted in the NBA',
 'Birthday': "Birthdate (YYYY-MM-DD)", 
 'Birthplace': "If in the United States, Contains State Name and Occasionally 'U.S.'; Else Contains Country Name and Occasionally a City Name", 
 'CareerStart': "First Year Playing in the NBA", 
 'CareerEnd': "Last Year Playing in the NBA",
 'Height (in)': "Player's Height Measured in Inches", 
 'Weight (lbs)': "Player's Weight Measured in Pounds", 
 'HS1':"Name of 1st High School Player Went to",
 'HS2':"Name of 2nd High School Player Went to",
 'HS3':"Name of 3rd High School Player Went to",
 'HS4':"Name of 4th High School Player Went to", 
 'HSC1':"City of 1st High School Player Went to",
 'HSC2':"City of 2nd High School Player Went to",
 'HSC3':"City of 3rd High School Player Went to", 
 'HSC4':"City of 4th High School Player Went to",
 'USHS':"1 if All the Player's High School Years Were in the U.S.; else 0",
 'CollegeY1':"Years Player Went to His 1st College",
 'CollegeY2':"Years Player Went to His 2nd College",
 'CollegeY3':"Years Player Went to His 3rd College",
 'CollegeY4':"Years Player Went to His 4th College",
 'College1':"1st College a Player Went to", 
 'College2':"2nd College a Player Went to",
 'College3':"3rd College a Player Went to",
 'College4':"4th College a Player Went to",
 'Teams':'Years, Team That NBA Player Was On'}

In [ ]:
cleaned_info["DataType"] = final_cleaned.dtypes
cleaned_info["DataType"] = cleaned_info["DataType"].apply(lambda x: 'str, NoneType' if x=="object" else x)
cleaned_info.loc["Teams","DataType"] = 'str, [str], NoneType'
cleaned_info["Description"] = descriptions.values()

In [ ]:
cleaned_info

In [ ]:
cleaned_info.to_csv('C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\nba_personal_info_cleaned_descriptions.csv')

In [ ]:
nba_stats_copy[nba_stats_copy['FirstName']=='Shaquille']